## Techport API

In [71]:
import pandas as pd
import requests

sinceDate = '2024-03-15'

resProjects = requests.get('https://techport.nasa.gov/api/projects?updatedSince='+sinceDate).json()
dfProjects = pd.DataFrame(resProjects['projects'])
dfProjects.head()

,acronym,projectId,title,website,lastUpdated
0,,116277,,,2024-3-28
1,,154531,,,2024-3-26
2,,154382,,,2024-3-26
3,,94884,,,2024-3-25
4,,147003,,,2024-3-25


In [72]:
dfProject = pd.DataFrame(
    columns=[
        'project_id', 
        'acronym', 
        'title', 
        'benefits', 
        'description', 
        'start_year', 
        'start_month', 
        'end_year', 
        'end_month', 
        'status_description',
        'website',
        'organization_id',
        'last_updated',
        'release_status_string',
    ]
)

dfOrganization = pd.DataFrame(
    columns=[
        'organization_id',
        'organization_name',
        'organization_type',
        'organization_type_pretty',
        'acronym',
        'city'
    ]
)
        
indexProject = 0
indexOrganization = 0

for index, row in dfProjects.iterrows():

    projectFound = dfProject.loc[dfProject['project_id'] == row['projectId']]

    #add project detail if not exists
    if len(projectFound) == 0:
        resProject = requests.get('https://techport.nasa.gov/api/projects/'+str(row['projectId'])).json()

        #append project detail to dfProject
        dfProject.loc[indexProject] = [
            resProject['project']['projectId'],
            resProject['project']['acronym'],
            resProject['project']['title'],
            resProject['project']['benefits'],
            resProject['project']['description'],
            resProject['project']['startYear'],
            resProject['project']['startMonth'],
            resProject['project']['endYear'],
            resProject['project']['endMonth'],
            resProject['project']['statusDescription'],
            resProject['project']['website'],
            resProject['project']['responsibleMd']['organizationId'],
            resProject['project']['lastUpdated'],
            resProject['project']['releaseStatusString']
        ]

        projectFound = dfProject.loc[indexProject]

        indexProject += 1
        
    #update empty cells in projects df
    dfProjects.at[index, 'acronym'] = projectFound['acronym']
    dfProjects.at[index, 'title'] = projectFound['title']
    dfProjects.at[index, 'website'] = projectFound['website']
    
    #add organization detail if not exists
    organizationFound = dfOrganization.loc[dfOrganization['organization_id'] == projectFound['organization_id']]

    if len(organizationFound) == 0:
        resOrganization = requests.get('https://techport.nasa.gov/api/organizations/'+str(projectFound['organization_id'])).json()

        #append organization detail to dfOrganization
        dfOrganization.loc[indexOrganization] = [
            resOrganization['organization']['organizationId'],
            resOrganization['organization']['organizationName'],
            resOrganization['organization']['organizationType'],
            resOrganization['organization']['organizationTypePretty'],
            resOrganization['organization']['acronym'],
            resOrganization['organization']['city']
        ]

        indexOrganization += 1
    

In [73]:
dfProjects.head()

,acronym,projectId,title,website,lastUpdated
0,PSI,116277,Plume Surface Interaction,,2024-3-28
1,,154531,Virtual Explanation Reasoning Agent (VERA),,2024-3-26
2,,154382,Abrasion Resistant and Flame-Resistant Textile...,,2024-3-26
3,NPAS,94884,NASA Platform for Autonomous Systems,https://www.nasa.gov/stennis/ssc-autonomous-sy...,2024-3-25
4,RAMPT-RAAMBO,147003,RAMPT - Refractory Alloy Additive Manufacturin...,,2024-3-25


In [74]:
dfProject.head()

,project_id,acronym,title,benefits,description,start_year,start_month,end_year,end_month,status_description,website,organization_id,last_updated,release_status_string
0,116277,PSI,Plume Surface Interaction,"<p>Develops an integrated modeling, simulation...",<p>Plume-surface interaction (PSI) describes t...,2020,5,2024,9,Active,,4875,2024-3-28,Released
1,154531,,Virtual Explanation Reasoning Agent (VERA),"For NASA, VERA would help astronauts in deep s...",On-board autonomy will be critical to the succ...,2023,7,2025,7,Active,,4875,2024-3-26,Released
2,154382,,Abrasion Resistant and Flame-Resistant Textile...,The proposed textiles will serve as a protecti...,Abrasion and flame-resistant yarn and textiles...,2023,8,2024,9,Active,,4875,2024-3-26,Released
3,94884,NPAS,NASA Platform for Autonomous Systems,<p>NPAS software is being cultivated as a para...,<p>Autonomous operations are critical for the ...,2016,9,2024,9,Active,https://www.nasa.gov/stennis/ssc-autonomous-sy...,9525,2024-3-25,Released
4,147003,RAMPT-RAAMBO,RAMPT - Refractory Alloy Additive Manufacturin...,<p> </p><p> </p><p> </p><p>RAAMBO benefits inc...,<p> </p><p> </p><p> </p><p>The Refractory Allo...,2021,10,2024,9,Active,,4875,2024-3-25,Released


In [75]:
dfOrganization.head()

,organization_id,organization_name,organization_type,organization_type_pretty,acronym,city
0,4875,Space Technology Mission Directorate,NASA_Mission_Directorate,NASA Mission Directorate,STMD,
1,9525,Exploration Systems Development Mission Direct...,NASA_Mission_Directorate,NASA Mission Directorate,ESDMD,
2,4910,Mission Support Directorate,NASA_Mission_Directorate,NASA Mission Directorate,MSD,
3,4909,Science Mission Directorate,NASA_Mission_Directorate,NASA Mission Directorate,SMD,


## Redshift

In [76]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

REDSHIFT_HOST =  os.getenv('REDSHIFT_HOST')
REDSHIFT_PORT = os.getenv('REDSHIFT_PORT')
REDSHIFT_DATABASE = os.getenv('REDSHIFT_DATABASE')
REDSHIFT_USER = os.getenv('REDSHIFT_USER')
REDSHIFT_PASSWORD = os.getenv('REDSHIFT_PASSWORD')

try:
    conn = psycopg2.connect(
        host=REDSHIFT_HOST,
        dbname=REDSHIFT_DATABASE,
        user=REDSHIFT_USER,
        password=REDSHIFT_PASSWORD,
        port=REDSHIFT_PORT
    )

    print("- conectado a redshift")
except Exception as e:
    print(e)
    print("- no se pudo conectar a redshift")

- conectado a redshift


In [77]:
#drop tables
with conn.cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS project")
    cursor.execute("DROP TABLE IF EXISTS organization")
    cursor.execute("DROP TABLE IF EXISTS projects")
    conn.commit()

In [78]:
#create tables
with conn.cursor() as cursor:
    tblProjects = """
    CREATE TABLE IF NOT EXISTS projects (
        project_id INT NOT NULL,
        acronym VARCHAR(50),
        title VARCHAR(250),
        website VARCHAR(250),
        last_updated DATE,
        etl_load DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    """
    cursor.execute(tblProjects)

    tblProject = """
    CREATE TABLE IF NOT EXISTS project (
        project_id INT NOT NULL,
        acronym VARCHAR(50),
        title VARCHAR(250),
        benefits VARCHAR(MAX),
        description VARCHAR(MAX),
        start_year VARCHAR(4),
        start_month VARCHAR(2),
        end_year VARCHAR(4),
        end_month VARCHAR(2),
        status_description VARCHAR(20),
        website VARCHAR(250),
        organization_id INT,
        last_updated DATE,
        release_status_string VARCHAR(20),
        etl_load DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    """
    cursor.execute(tblProject)

    tblOrganization = """
    CREATE TABLE IF NOT EXISTS organization (
        organization_id INT NOT NULL,
        organization_name VARCHAR(250),
        organization_type VARCHAR(250),
        organization_type_pretty VARCHAR(250),
        acronym VARCHAR(50),
        city VARCHAR(100),
        etl_load DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    """
    cursor.execute(tblOrganization)       

    conn.commit()

In [79]:
from psycopg2.extras import execute_values

# insert data in tbl projects
with conn.cursor() as cursor:
    execute_values(
        cursor,
        """
        INSERT INTO projects (
            acronym,
            project_id,
            title,
            website,
            last_updated
        ) VALUES %s
        """,
        [tuple(row) for row in dfProjects.values],
        page_size=len(dfProjects)
    )
    conn.commit()

In [80]:
# insert data in tbl project
with conn.cursor() as cursor:
    execute_values(
        cursor,
        """
        INSERT INTO project (
            project_id,
            acronym,
            title,
            benefits,
            description,
            start_year,
            start_month,
            end_year,
            end_month,
            status_description,
            website,
            organization_id,
            last_updated,
            release_status_string
        ) VALUES %s
        """,
        [tuple(row) for row in dfProject.values],
        page_size=len(dfProject)
    )
    conn.commit()

In [81]:
# insert data in tbl organization
with conn.cursor() as cursor:
    execute_values(
        cursor,
        """
        INSERT INTO organization (
            organization_id,
            organization_name,
            organization_type,
            organization_type_pretty,
            acronym,
            city
        ) VALUES %s
        """,
        [tuple(row) for row in dfOrganization.values],
        page_size=len(dfOrganization)
    )
    conn.commit()